In [20]:
import statsmodels.api as sm
import pandas as pd
import re,string
import nltk
from patsy import dmatrices
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import warnings
%pylab inline
import numpy as np
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

Populating the interactive namespace from numpy and matplotlib


In [21]:
import pickle as pk

In [65]:
filename = "Scraper/pros"
infile = open(filename,'rb')
pros_list = pk.load(infile)
infile.close()

In [66]:
filename = "Scraper/cons"
infile = open(filename,'rb')
cons_list = pk.load(infile)
infile.close()

In [67]:
pros_clean = [re.sub(r'[^\w\s]', ' ', y.lower()) for x in pros_list for y in x]
pros_clean = [re.sub(r'[\s+\n+]', ' ', x) for x in pros_clean]

In [68]:
cons_clean = [re.sub(r'[^\w\s]', ' ', y.lower()) for x in cons_list for y in x]
cons_clean = [re.sub(r'[\s+\n+]', ' ', x) for x in cons_clean]

In [104]:
stop = set(stopwords.words('english'))
punc = string.punctuation
pros_tokenize = [word for x in pros_clean for word in word_tokenize(x) if word not in stop if word not in punc]
cons_tokenize = [word for x in cons_clean for word in word_tokenize(x) if word not in stop if word not in punc]

In [63]:
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer # need to do 'pip3 install twython' for this to work properly
from nltk.collocations import *

C:\Users\abhin\Anaconda2\envs\Python3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [64]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [70]:
proFreqs = nltk.FreqDist(pros_tokenize)
conFreqs = nltk.FreqDist(cons_tokenize)

In [72]:
proFreqs.most_common(10)

[('work', 1103),
 ('great', 705),
 ('good', 679),
 ('benefits', 587),
 ('pay', 507),
 ('reviews', 505),
 ('get', 479),
 ('people', 412),
 ('working', 406),
 ('lot', 389)]

In [73]:
conFreqs.most_common(10)

[('work', 1302),
 ('life', 676),
 ('balance', 643),
 ('reviews', 504),
 ('hours', 342),
 ('get', 325),
 ('cons', 315),
 ('time', 308),
 ('management', 288),
 ('worked', 287)]

In [74]:
confinder2 = BigramCollocationFinder.from_words(cons_tokenize)
confinder2.apply_freq_filter(10)
contop10_2grams = confinder2.nbest(bigram_measures.pmi, 10)
con_scored_2 = confinder2.score_ngrams(bigram_measures.pmi)
con_sorted = sorted(con_scored_2, key = lambda x: x[1])
contop20_2grams_sorted = con_sorted[-20:]
print(contop20_2grams_sorted)
with open('cons_wordle.txt', 'w') as f:
    for item in contop20_2grams_sorted:
        f.write(' '.join(item[0]) + ':' + str(item[1]) + '\n')

In [92]:
confinder2.nbest(bigram_measures.pmi, 10)

[('mandatory', 'overtime'),
 ('physically', 'demanding'),
 ('peak', 'season'),
 ('6', 'months'),
 ('fast', 'paced'),
 ('leadership', 'principles'),
 ('tier', '1'),
 ('night', 'shift'),
 ('even', 'though'),
 ('short', 'breaks')]

In [82]:
maxW = max([gram[1] for gram in contop20_2grams_sorted])
minW = min([gram[1] for gram in contop20_2grams_sorted])
contop20 = []
for gram in contop20_2grams_sorted:
    contop20.append([gram[0], (gram[1]-minW+5)/(maxW-minW)])

with open('cons 2 grams worlde.txt', 'w') as writeFile:
    for gram in contop20:
        writeFile.write(' '.join(gram[0]) + ':' + str(gram[1]) + '\n')

In [111]:
def bigram_maker(tokenizedList,filename,num):
    bg = BigramCollocationFinder.from_words(tokenizedList)
    bg.apply_freq_filter(10)
    bg_measures = bg.score_ngrams(bigram_measures.pmi)
    bg_sorted = sorted(bg_measures, key = lambda x: x[1])
    bg_sorted_20 = bg_sorted[-num:][::-1]
    print("Top 20 bigrams: \n")
    print(bg_sorted_20)
    with open(filename, 'w') as f:
        for item in bg_sorted_20:
            f.write(' '.join(item[0]) + ':' + str(item[1]) + '\n')

In [108]:
#bigram_maker(cons_tokenize,"cons_word.txt")
bigram_maker(pros_tokenize,"pros_bigram_word.txt",10)
bigram_maker(cons_tokenize,"cond_bigram_word.txt",10)

Top 20 bigrams: 

[(('dress', 'code'), 10.683955913757938), (('cutting', 'edge'), 10.498089368446605), (('laid', 'back'), 10.135519289061897), (('minimum', 'wage'), 9.890406791225367), (('decision', 'making'), 9.479473690279258), (('co', 'workers'), 9.189967073084278), (('dental', 'vision'), 9.032425796097796), (('10', 'hour'), 8.355484973003804), (('customer', 'obsession'), 8.307758156342455), (('leadership', 'principles'), 8.250996506481835)]
Top 20 bigrams: 

[(('mandatory', 'overtime'), 9.378230365061858), (('physically', 'demanding'), 8.712848162002643), (('peak', 'season'), 8.694392614257897), (('6', 'months'), 8.629297586036012), (('fast', 'paced'), 8.607062367561289), (('leadership', 'principles'), 8.292869921453534), (('tier', '1'), 8.261854831896276), (('night', 'shift'), 8.036721901204979), (('even', 'though'), 7.727272745599308), (('short', 'breaks'), 7.635433162826663)]


In [109]:
def trigram_maker(tokenizedList,filename,num):
    tg = TrigramCollocationFinder.from_words(tokenizedList)
    tg.apply_freq_filter(10)
    tg_measures = tg.score_ngrams(trigram_measures.pmi)
    tg_sorted = sorted(tg_measures, key = lambda x: x[1])
    tg_sorted_20 = tg_sorted[-num:][::-1]
    print("Top 20 Trigrams: \n")
    print(tg_sorted_20)
    with open(filename, 'w') as f:
        for item in tg_sorted_20:
            f.write(' '.join(item[0]) + ':' + str(item[1]) + '\n')

In [114]:
print("******pros******")
trigram_maker(pros_tokenize,"pros_trigram_word.txt",20)
print("******cons******")
trigram_maker(cons_tokenize,"cons_trigram_word.txt",20)

******pros******
Top 20 Trigrams: 

[(('4', 'days', 'week'), 14.113308873651796), (('short', 'duration', '500'), 13.753685109500257), (('life', 'balance', '520'), 13.271409689390381), (('lot', 'short', 'duration'), 13.127338688013978), (('pros', 'working', 'towards'), 13.020705720103699), (('working', 'towards', 'better'), 12.931976801509368), (('duration', '500', 'reviews'), 12.823394081311662), (('balance', '520', 'reviews'), 12.57667233061347), (('learn', 'lot', 'short'), 12.530880548454991), (('fast', 'paced', 'environment'), 12.34763639344867), (('time', 'unpaid', 'time'), 12.177929173732313), (('520', 'reviews', 'get'), 11.896792159075392), (('get', 'learn', 'lot'), 11.67106489957537), (('towards', 'better', 'work'), 11.49009564310316), (('work', 'life', 'balance'), 11.358949468221244), (('reviews', 'get', 'learn'), 11.294551666922406), (('better', 'work', 'life'), 11.193378111013079), (('day', 'work', 'week'), 10.541862010920287), (('benefits', 'day', '1'), 10.520876122175636), 

Interesting frequent words - bigrams and tri grams

* Lift scores - positive and negative attributes
* Topic Modeling
* Sentiment
* Cosine Similarity with mission statement
* Conditional Random Fields

* Rating vs. word length
* Recommendation by department, location
* Titles - Most frequent words in title review
* Most frequent words in highly rated reviews
* Most frequent words in low rated reviews
* Plot sentiments with time, anything stands out
* Plot sentiments across job titles, location - US Map of sentiments

Sentiment, topics, word length, frequent words across:
* High / low ratings - reviews
* CEO
* Outlook
* Recommends
* Title - for high/low ratings
* Job function
* Location
* Current / Former employee